In [9]:
!pip install langchain langgraph cassio

In [10]:
!pip install langchain_community

In [11]:
!pip install -U tiktoken langchain-groq langchainhub chromadb langchain langgraph langchain_huggingface

In [15]:
!pip install --upgrade astrapy

  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached h2-4.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached dnspython-2.6.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached hyperframe-6.0.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached hpack-4.0.0-py3-none-any.whl.metadata (2.5 kB)
Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
   ---------------------------------------- 0.0/680.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/680.4 kB ? eta -:--:--
   --------------- ------------------------ 262.1/680.4 kB ? eta -:--:--
   ------------------------------ --------- 524.3/680.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 680.4/680.4 kB 1.7 MB/s eta 0:00:00
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
Using cached dnspython-2.6.1-py3-none-any.whl (307 kB)
Using cached h2-4.1.0-py3-none-any.whl (57 kB)
Using cached hpac

In [17]:
#Connecting Vector DB
import cassio


import os 
from dotenv import load_dotenv

load_dotenv()
## connection of the ASTRA DB
ASTRA_DB_APPLICATION_TOKEN= os.getenv("ASTRA_DB_APPLICATION_TOKEN")# enter the "AstraCS:..." string found in in your Token JSON file"
ASTRA_DB_ID= os.getenv("ASTRA_DB_ID")
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)


DependencyException: Unable to load a default connection class
The following exceptions were observed: 
 - The C extension needed to use libev was not found.  This probably means that you didn't have the required build dependencies when installing the driver.  See http://datastax.github.io/python-driver/installation.html#c-extensions for instructions on installing build dependencies and building the C extension.
 - Unable to import asyncore module.  Note that this module has been removed in Python 3.12 so when using the driver with this version (or anything newer) you will need to use one of the other event loop implementations.